In [22]:
%%writefile app.py
import streamlit as st
from google import genai

# --- Load knowledge base ---
with open("knowledge.txt", "w", encoding="utf-8") as f:
    f.write("This is a sample Arduino knowledge base. Replace this with your real file!")

with open("knowledge.txt", "r", encoding="utf-8") as f:
    knowledge_base = f.read()

# --- Mock secrets for Colab ---
st.secrets["GEMINI_API_KEY"] = "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"

# --- Session state ---
if "chat_history" not in st.session_state:
    st.session_state.chat_history = [
        {"role": "system", "content": "👋 Welcome to Arduino Expert! Ask me anything about Arduino projects."}
    ]

def chat(user_input):
    global knowledge_base
    api_key = st.secrets["GEMINI_API_KEY"]

    if not api_key:
        return "⚠️ Please set your Gemini API key."

    context_text = f"Knowledge base:\n{knowledge_base}\n\n"
    for msg in st.session_state.chat_history:
        context_text += f"{msg['role'].capitalize()}: {msg['content']}\n"
    context_text += f"User: {user_input}\nAnswer based on the knowledge above."

    try:
        client = genai.Client(api_key=api_key)
        chat_session = client.chats.create(model="gemini-2.5-flash")
        response = chat_session.send_message(context_text)
        reply = response.text
    except Exception as e:
        reply = f"❌ Error: {e}"

    st.session_state.chat_history.append({"role": "user", "content": user_input})
    st.session_state.chat_history.append({"role": "assistant", "content": reply})
    return reply

st.set_page_config(page_title="Arduino Expert", page_icon="🤖")

st.markdown("<h2 style='text-align:center;color:#0082C9;'>🤖 Arduino Expert Chatbot</h2>", unsafe_allow_html=True)

for msg in st.session_state.chat_history:
    if msg["role"] == "user":
        st.chat_message("user").markdown(msg["content"])
    elif msg["role"] == "assistant":
        st.chat_message("assistant").markdown(msg["content"])
    elif msg["role"] == "system":
        st.chat_message("system").markdown(msg["content"])

user_input = st.chat_input("Ask a question...")
if user_input:
    reply = chat(user_input)
    st.chat_message("assistant").markdown(reply)

if st.button("🧹 Clear Chat"):
    st.session_state.chat_history = [
        {"role": "system", "content": "👋 Welcome to Arduino Expert! Ask me anything about Arduino projects."}
    ]
    st.experimental_rerun()


Overwriting app.py
